# Using Whisper for podcast transcription using RSS feeds
This notebook provides a example of using FeedParser with OpenAI's Whisper.

[FeedParser](https://feedparser.readthedocs.io/en/latest/) is a Python library for downloading and parsing syndicated feeds including RSS, Atom & RDF Feeds.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1RjVE2zPYY6FFAL3k0S4AgZJYmoV0uH6w)
## Install required Python packages
Run the cell below to install Whisper, FFMPEG (If not installed), and FeedParser.

The Python libraries `openai`, `cohere`, and `tiktoken` are also installed because of dependencies for the `llmx` library. That is because `llmx` relies on them to function correctly.

In [ ]:
!pip install -q cohere openai tiktoken
!apt-get install ffmpeg
!pip install -q "git+https://github.com/openai/whisper.git"
!pip install -q feedparser requests

In [ ]:
import feedparser
import requests
import os
import time
from urllib.parse import urlparse
import subprocess
import re

In [ ]:
def list_episodes(feed_url):
    """
    Lists all episodes in the given RSS feed.

    Parameters:
    - feed_url: The RSS feed URL.

    Returns:
    - A list of tuples containing episode titles, their URLs and published dates in 'YYYYMMDD' format.
    """
    d = feedparser.parse(feed_url)

    episodes = []
    for entry in d.entries:
        title = entry.title
        published = time.strftime('%Y%m%d', time.gmtime(time.mktime(entry.published_parsed)))
        url = None
        for link in entry.links:
            if link.type == "audio/mpeg":
                url = link.href
                break
        if url:
            episodes.append((title, url, published))

    return episodes

from urllib.parse import urlparse

def download_episode(url, filename=None):
    """
    Downloads the podcast episode from the given URL.

    Parameters:
    - url: The URL of the podcast episode.
    - filename: The desired filename to save the podcast. If not provided, it'll use the last part of the URL.

    Returns:
    - The path to the downloaded file.
    """
    # If a custom filename is provided, append the appropriate extension from the URL
    if filename:
        parsed_url = urlparse(url)
        # Extract only the base path without any query parameters
        base_path = os.path.basename(parsed_url.path)
        ext = os.path.splitext(base_path)[1]
        filename += ext
    else:
        filename = os.path.basename(parsed_url.path)

    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    return filename

def download_episode_start_at(url, filename=None, start_at=0):
    """
    Downloads the podcast episode from the given URL and trims it starting from 'start_at' seconds.

    Parameters:
    - url: The URL of the podcast episode.
    - filename: The desired filename to save the podcast. If not provided, it'll use the last part of the URL.
    - start_at: The start time in seconds from where the audio should be trimmed.

    Returns:
    - The path to the downloaded and trimmed file.
    """
    parsed_url = urlparse(url)
    if filename:
        # Ensure the filename has the correct extension
        ext = os.path.splitext(parsed_url.path)[1]
        filename += ext
    else:
        filename = os.path.basename(parsed_url.path)

    # Download the file
    response = requests.get(url, stream=True)
    response.raise_for_status()

    temp_filename = "temp_" + filename
    with open(temp_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    # Use ffmpeg to trim the audio file
    trimmed_filename = "trimmed_" + filename
    command = ['ffmpeg', '-y', '-i', temp_filename, '-ss', str(start_at), '-c', 'copy', trimmed_filename]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Remove the original downloaded file
    os.remove(temp_filename)

    return trimmed_filename

## RSS feed podcast

In [ ]:
# Science, Quickly
podcast = 'https://feeds.megaphone.fm/SAM8799470961'
# WSJ Minute Briefing
# podcast = 'https://feeds.megaphone.fm/WSJ7928321669'

d = feedparser.parse(podcast)
print(f"Podcast name:", d.feed.title)
print(f"Number of episodes:", len(d.entries))

# List episodes
episodes = list_episodes(podcast)
# Print the first 10 episodes
print("Episodes:")
for idx, (title, url, published) in enumerate(episodes, 1):
    print(f"{idx}. {published}-{title}")
    if idx == 10:
        break

In [ ]:
episode_num = 5 #@param {type:"integer"}
drive_folder = "" #@param {type:"string"}

title, url, published = episodes[episode_num - 1]
custom_filename = published + '-' + (re.sub('[^A-Za-z0-9 ]+', '', title[:75]).replace(' ', '_'))

# Download the selected episode
audio_file = download_episode_start_at(url, drive_folder + custom_filename, 30)
print(f"Downloaded '{title}' as {audio_file}.")

In [ ]:
import ipywidgets as widgets
widgets.Audio.from_file(audio_file, autoplay=False, loop=False)

In [ ]:
import whisper
import torch
# NLTK helps to split the transcription sentence by sentence
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

model = whisper.load_model("small")

audio = whisper.load_audio(audio_file)
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# decode the audio
options = whisper.DecodingOptions(fp16=torch.cuda.is_available(), task='transcribe')
result = whisper.decode(model, mel, options)

# print the recognized text
print("----\nTranscription from audio:")
for sent in sent_tokenize(result.text):
  print(sent)